<a href="https://colab.research.google.com/github/Sara-Samer/sentiment-analsis-with-TF-IDF/blob/main/Sentiment_analsis_with_TF_IDF_and_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data manipulation

## Google sheet setup
---
To work with google sheet directly without downloading the dataset  


google sheet link *(please click on this link first then run the notebook)*: [sheet](https://docs.google.com/spreadsheets/d/1rrlGMui0b2Xniac-YjUtnpq3KapdkXJtFaWEdkdtCNI/edit?usp=sharing)

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
worksheet = gc.open('2k_reviews').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values() # we will use this later is importing data

## imports

In [ ]:
!pip install contractions

     |████████████████████████████████| 286kB 2.8MB/s 
     |████████████████████████████████| 327kB 32.1MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85400 sha256=30d9fc480a7d7be63ad13739b96a35c11acfa7a39f40fb15612a09c46aa9716a
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [ ]:
import pandas as pd
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import contractions
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Import data

In [ ]:
df = pd.DataFrame.from_records(rows[1:], columns=['review', 'sentiment'])
df.head()

,review,sentiment
0,"the year is barely a week old , and there is a...",0
1,those of you who frequently read my reviews ar...,1
2,the promotion for fear and loathing in las veg...,1
3,"synopsis : original "" jurassic park "" survivor...",0
4,it stands as a moment one will not soon forget...,1


# TF-IDF

## TF-IDF analyzer

In [ ]:
def analyzer(text):
    # Clean HTML tags
    filtered_text = BeautifulSoup(text).get_text()
    # Expand contractions
    filtered_text = contractions.fix(filtered_text)
    # Remove punctuation
    # filtered_text = re.sub("[^a-zA-Z]"," ", filtered_text)
    filtered_text = re.sub(r'/', ' ', filtered_text)
    filtered_text = re.sub(r'[^\w\s]', '', filtered_text)
    # Tokenize
    words = filtered_text.lower().split()
    # Remove stopwords
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w, pos="v") for w in words]
    words = [lemmatizer.lemmatize(w, pos="n") for w in words]
    # Stemming
    porter_stemmer = PorterStemmer()
    words = [porter_stemmer.stem(w) for w in words]
    return words


In [ ]:
analyzer("He'd , a young child, was running/walking and/or eating at same time -bad comment at the time :)-. He has bad habit of swimming after playing long hours in the Sun.")

['would',
 'young',
 'child',
 'run',
 'walk',
 'eat',
 'time',
 'bad',
 'comment',
 'time',
 'bad',
 'habit',
 'swim',
 'play',
 'long',
 'hour',
 'sun']

## TF-IDF vectorizer

In [ ]:
vectorizer = TfidfVectorizer(analyzer=analyzer) 
tfidf = vectorizer.fit_transform(df['text']) 
y =  df['sentiment']

# Regression model

## test and train divide

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(tfidf,y,test_size=0.3,random_state=0)

## Logistic regression model

In [ ]:
model = LogisticRegressionCV(cv=5, scoring='accuracy', random_state=0, n_jobs=-1, verbose=3, max_iter=300).fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   13.5s finished


# Test

In [ ]:
model.score(X_test,y_test)

0.8283333333333334

In [ ]:
#to predict
query = ["Worst movie I have ever seen in my entire life",
         "greatest plot ever, it was a master piece",
         "The movie was meh, not much to be a fan of",
         "I liked it a lot, the hero was smart and the female lead was amazing"]

features = vectorizer.transform(query)
model.predict(features)

array([0, 1, 0, 1])